In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import pyeCAP
import numpy as np
import pandas as pd
import scipy.io as sio
from scipy import signal
import matplotlib.pyplot as plt
import traceback
import os
import dask.array as da
import plotly.express as px

In [ ]:
"Import meta data"
"""Get HF meta data from excel file"""
excel_path = r'E:\HF_Block\20240904_HFBlock_Pig05\20240904_HFBlock_05_Parameters_modified.xlsx'
metaDF = pd.read_excel(excel_path, sheet_name = 'HF05_metaDF')

"""Remove rows that are not needed for current analysis and reset index of metaDF"""
metaDF.drop(metaDF.loc[ metaDF['Type'] == 'HF Only'].index, inplace = True)  #Removed HF only tanks as they did not have any stim parameters
metaDF.drop(metaDF.loc[ metaDF['Type'] == 'Control'].index, inplace = True)
metaDF.drop(metaDF.loc[ metaDF['Type'] == 'X'].index, inplace = True)
metaDF.drop(metaDF.loc[ metaDF['Type'] == 'Thresholding'].index, inplace = True)
metaDF.drop(metaDF.loc[ metaDF['Type'] == 'Contact ID'].index, inplace = True)
metaDF.reset_index(inplace = True)

"TDT Data Import - List"
tdt_path = r'E:\HF_Block\20240904_HFBlock_Pig05\HF_Block_Template-240904-083702' #Lab Path
#tdt_path = r'G:\Data\HF_Block\20240904_HFBlock_Pig05\HF_Block_Template-240904-083702' #Local Path 
tdt_file_list_all = os.listdir(tdt_path)
tdt_file_list_all.remove('desktop.ini')

ADI_path = r'E:\HF_Block\20240904_HFBlock_Pig05' #Lab Path
#ADI_path = r'G:\Data\HF_Block\20240904_HFBlock_Pig05' #Local path
ADI_file = r'\20240904_HFBlock_Pig05_stats_modified.mat'

raw_phys = pyeCAP.Phys(ADI_path + ADI_file)
ws_stripped_names = [i.strip() for i in raw_phys.ch_names] #This line just removes the white spaces around the names imported
raw_phys = raw_phys.set_ch_names(ws_stripped_names)

"""Ephys data instantiation"""
tdt_path_list = [tdt_path + '\\' + fileNAME for fileNAME in metaDF['TDT Tank']]

#Data Streams
raw_ECAP = pyeCAP.Ephys(tdt_path_list, stores = 'ECAP')
raw_ECAP = raw_ECAP.remove_ch(['ECAP 6', 'ECAP 7', 'ECAP 8'])

"""Changes the stim multi-index to utilize the Tank names from tdt_file_list instead [0,1,2,3...]"""
stim = pyeCAP.Stim(tdt_path_list)
stimDF = stim.parameters
#tank_to_params_dict = { tank:params for (tank,params) in zip(metaDF['TDT Tank'].to_list(), stim.parameters.groupby(level=0).groups.values() ) }
#params_to_tank_dict = { param:tank for (param,tank) in zip(stim.parameters.groupby(level=0).groups.keys(), metaDF['TDT Tank'].to_list()) }

"""ECAP/EMG/TriCAP object instantiation"""
ECAPr = pyeCAP.ECAP(raw_ECAP, stim)
PHYS = pyeCAP.ECAP(raw_phys,stim)
raw_ECG_array = np.squeeze( raw_phys.array[raw_phys._ch_to_index('EKG Raw')].compute() )

In [ ]:
"""Band-pass Filter"""
ECG_fs = 1000
nyq_rate = ECG_fs / 2

cutoffs = [5,15]
sos = signal.butter(4, cutoffs, btype='bandpass', fs = ECG_fs, output="sos")
a, b = signal.sos2tf(sos)
overlap = max(len(a), len(b))
print(overlap)

"""Code which replaces any NaN values by simply replacing them with the first value after the NaN's occur"""
#nan_idx = np.argwhere(np.isnan(raw_ECG))
#np.nan_to_num(raw_ECG_array, copy=False, nan=raw_ECG_array[nan_idx[-1] + 1])

ECG_BP_filt = signal.sosfiltfilt(sos, raw_ECG_array)
ECG_dvt_filt = np.diff(ECG_BP_filt,append=0)
#ECG_sq = np.square(ECG_dvt_filt)
#ECG_moving_average = uniform_filter1d(ECG_sq,size=150)

In [ ]:
fs = raw_phys.sample_rate
transition_width = 0.5
filter_cutoffs = [5, 15]
numtaps = 3000
filter_weights = signal.firwin(numtaps, filter_cutoffs, width=transition_width, window='Hamming',
                                       pass_zero='bandpass', fs=fs)
ECG_BP_filt_fir = signal.fftconvolve(np.flip(signal.fftconvolve(raw_ECG_array, filter_weights, mode="same")), filter_weights,mode="same",)
ECG_dvt_filt_fir = np.diff(ECG_BP_filt_fir,append=0)

In [ ]:
filt_data, data_diff_sq,data_moving_average = raw_phys.pan_tompkins_algorithm(ecg_ch_name = 'EKG Raw')

In [ ]:
fig, ax = plt.subplots(figsize = (15,10))
ax.plot()
plt.show()

In [ ]:
raw_phys.sample_rate

In [ ]:
raw_phys.time()[0:100].compute()

In [ ]:
data = raw_phys.pan_tompkins_algorithm(ecg_ch_name = 'EKG Raw')

In [ ]:
filt_data, data_diff_sq,data_moving_average = raw_phys.pan_tompkins_algorithm(ecg_ch_name = 'EKG Raw')

In [ ]:
new_array = [da.squeeze(da.stack([d1, d2, d3])) for d1,d2,d3 in zip(filt_data,data_diff_sq,data_moving_average)]#, data_diff_sq, data_moving_average)

In [ ]:
new_array

In [ ]:
new_array = [da.concatenate([d1, d2, d3]) for d1,d2,d3 in zip(filt_data,data_diff_sq,data_moving_average)]#, data_diff_sq, data_moving_average)

In [ ]:
combined_array = [da.concatenate([d1,d2], axis=0) for d1,d2 in zip(raw_phys.data,new_array)]

In [ ]:
combined_array

In [ ]:
filt_data_array = np.squeeze(da.concatenate(filt_data, axis = 1).compute())
diff_sq_array = np.squeeze(da.concatenate(data_diff_sq, axis = 1).compute())
moving_average_array = np.squeeze(da.concatenate(data_moving_average, axis = 1).compute())

In [ ]:
"""
Finds peaks based on parameters height and a minimum distance between peaks
"""
peaks = signal.find_peaks(moving_average_array, height = np.mean(moving_average_array), distance = raw_phys.sample_rate * 0.25)[0]

In [ ]:
peaks_in_range = np.where( (peaks >= plot_start_idx) & (peaks <= plot_stop_idx))

In [ ]:
plt.close('all')
plt.clf()

plot_start_time_sec = 5770
plot_stop_time_sec = 5775

plot_start_idx = plot_start_time_sec * 1000
plot_stop_idx = plot_stop_time_sec * 1000

#fig, ax = plt.subplots(figsize = (15,10))
#ax.plot(raw_phys.time()[plot_start_idx:plot_stop_idx].compute(), filt_data_array[plot_start_idx:plot_stop_idx])
#ax.plot(raw_phys.time()[plot_start_idx:plot_stop_idx].compute(), diff_sq_array[plot_start_idx:plot_stop_idx], color = 'r')
#ax.plot(raw_phys.time()[plot_start_idx:plot_stop_idx].compute(), moving_average_array[plot_start_idx:plot_stop_idx], color = 'g')
#plt.show()

fig, ax = plt.subplots(nrows = 3, ncols=1, figsize=(25,15))

"""
Upper plot of filtered data with calculated HR (bpm) overlaid
"""
ax[0].plot(raw_phys.time()[plot_start_idx:plot_stop_idx].compute(), filt_data_array[plot_start_idx:plot_stop_idx])

#get peaks within plot window
mw_stdev = np.std(moving_average_array)

#ax[0].scatter(raw_phys.time()[peaks], ,s=50, c='r', marker='x')
#ax[0].scatter(peaks[0] * time_per_sample, filtered_data[peaks[0]],s=50, c='r', marker='x')
#ax2 = ax[0].twinx()
#ax2.plot(hrDF['Time (sec)'], hrDF['HR (bpm)'], color='red')

"""
Middle plot shows the diff_sq data that peak indices were derived from. Red line indicates height threshold
"""
mw_stdev = np.std(moving_average_array)
#ax[1].plot(raw_phys.time()[plot_start_idx:plot_stop_idx].compute(), diff_sq_array[plot_start_idx:plot_stop_idx])
ax[1].plot(raw_phys.time()[plot_start_idx:plot_stop_idx].compute(), moving_average_array[plot_start_idx:plot_stop_idx], color = 'r')
#ax[1].axhline(mw_stdev, color='g')
plt.show()

In [ ]:
raw_phys.sample_rate

In [ ]:
filt_data, data_diff_sq,data_moving_average = raw_phys.pan_tompkins_algorithm(ecg_ch_name = 'EKG Raw')

filt_data_array = np.squeeze(da.concatenate(filt_data, axis = 1).compute())
diff_sq_array = np.squeeze(da.concatenate(data_diff_sq, axis = 1).compute())
moving_average_array = np.squeeze(da.concatenate(data_moving_average, axis = 1).compute())

fig, ax = plt.subplots(figsize = (15,10))
#ax.plot(raw_phys.time().compute(), filt_data_array)
ax.plot(raw_phys.time()[5000000:].compute(), ECG_dvt_filt_fir[5000000:])
#ax.plot(raw_phys.time()[5000000:].compute(), diff_sq_array[5000000:], color = 'r')
#test = np.subtract(ECG_dvt_filt_fir, diff_sq_array)
#test2 = np.subtract(ECG_BP_filt_fir, filt_data_array)
#ax.plot(raw_phys.time().compute(), test, color = 'g')
#ax.plot(raw_phys.time().compute(), moving_average_array, color = 'g')
#ax.plot(raw_phys.time()[5000000:].compute(), ECG_dvt_filt[5000000:])
plt.show()

In [ ]:
filt_data, data_diff_sq,data_moving_average = raw_phys.pan_tompkins_algorithm(ecg_ch_name = 'EKG Raw')

filt_data_array = np.squeeze(da.concatenate(filt_data, axis = 1).compute())
diff_sq_array = np.squeeze(da.concatenate(data_diff_sq, axis = 1).compute())
moving_average_array = np.squeeze(da.concatenate(data_moving_average, axis = 1).compute())

fig, ax = plt.subplots(figsize = (15,10))
#ax.plot(raw_phys.time().compute(), filt_data_array)
ax.plot(raw_phys.time()[5000000:].compute(), ECG_dvt_filt[5000000:])
#ax.plot(raw_phys.time()[5000000:].compute(), diff_sq_array[5000000:], color = 'r')
test = np.subtract(ECG_dvt_filt, diff_sq_array)
ax.plot(raw_phys.time()[5000000:].compute(), test[5000000:], color = 'g')
#ax.plot(raw_phys.time().compute(), moving_average_array, color = 'g')
#ax.plot(raw_phys.time()[5000000:].compute(), ECG_dvt_filt[5000000:])
plt.show()

In [ ]:
filt_data, data_diff_sq,data_moving_average = raw_phys.pan_tompkins_algorithm(ecg_ch_name = 'EKG Raw')

filt_data_array = np.squeeze(da.concatenate(filt_data, axis = 1).compute())
diff_sq_array = np.squeeze(da.concatenate(data_diff_sq, axis = 1).compute())
moving_average_array = np.squeeze(da.concatenate(data_moving_average, axis = 1).compute())

fig, ax = plt.subplots(figsize = (15,10))
#ax.plot(raw_phys.time().compute(), filt_data_array)
ax.plot(raw_phys.time()[5000000:].compute(), ECG_dvt_filt[5000000:])
#ax.plot(raw_phys.time()[5000000:].compute(), diff_sq_array[5000000:], color = 'r')
#test = np.subtract(ECG_dvt_filt, diff_sq_array)
#ax.plot(raw_phys.time()[5000000:].compute(), test[5000000:], color = 'g')
#ax.plot(raw_phys.time().compute(), moving_average_array, color = 'g')
#ax.plot(raw_phys.time()[5000000:].compute(), ECG_dvt_filt[5000000:])
plt.show()

In [ ]:
np.array_equal(filt_data_array, ECG_BP_filt_fir)

In [ ]:
np.array_equal(diff_sq_array, ECG_dvt_filt)

In [ ]:
filt_data_array = np.squeeze(da.concatenate(filt_data, axis = 1).compute())
diff_sq_array = np.diff(filt_data_array, append=filt_data_array[-1])
#moving_average_array = np.squeeze(da.concatenate(data_moving_average, axis = 1).compute())

t_start = 17000 * 1000
t_stop = 18000 * 1000

fig, ax = plt.subplots(figsize = (15,10))
ax.plot(raw_phys.time()[t_start:t_stop].compute(), filt_data_array[t_start:t_stop])
ax.plot(raw_phys.time()[t_start:t_stop].compute(), diff_sq_array[t_start:t_stop], color = 'r')
#ax.plot(raw_phys.time().compute(), moving_average_array, color = 'g')
plt.show()

In [ ]:
filt_data_array = np.squeeze(da.concatenate(filt_data, axis = 1).compute())
diff_sq_array = np.diff(filt_data_array, append=filt_data_array[-1])
#moving_average_array = np.squeeze(da.concatenate(data_moving_average, axis = 1).compute())

t_start = 12395 * 1000
t_stop = 12400 * 1000

fig, ax = plt.subplots(figsize = (15,10))
ax.plot(raw_phys.time()[t_start:t_stop].compute(), filt_data_array[t_start:t_stop])
ax.plot(raw_phys.time()[t_start:t_stop].compute(), diff_sq_array[t_start:t_stop], color = 'r')
#ax.plot(raw_phys.time().compute(), moving_average_array, color = 'g')
plt.show()

In [ ]:
plotDF = pd.DataFrame()
t_start = 9500 * 1000
t_stop = 10000 * 1000
plotDF['Time (s)'] = raw_phys.time()[t_start:t_stop].compute()
plotDF['BP Filtered Data'] = filt_data_array[t_start:t_stop]
plotDF['Differentiated Data'] = diff_sq_array[t_start:t_stop]
plotDF['Raw ECG'] = raw_ECG_array[t_start:t_stop]

fig = px.line(plotDF, x = 'Time (s)', y=['BP Filtered Data', 'Differentiated Data'])#, 'Raw ECG'])
fig.update_traces(hovertemplate='%{x:.10f}%{y:.10f}')
fig.show()

In [ ]:
plotDF = pd.DataFrame()
t_start = 34900 * 1000
t_stop = 35100 * 1000
plotDF['Time (s)'] = raw_phys.time()[t_start:t_stop].compute()
plotDF['BP Filtered Data'] = ECG_BP_filt_fir[t_start:t_stop]
plotDF['Differentiated Data'] = ECG_dvt_filt_fir[t_start:t_stop]
#plotDF['Raw ECG'] = raw_ECG_array[t_start:t_stop]

fig = px.line(plotDF, x = 'Time (s)', y=['BP Filtered Data', 'Differentiated Data'])#, 'Raw ECG'])
fig.update_traces(hovertemplate='%{x:.10f}')
fig.show()

In [ ]:
b,a = signal.butter(10, [5, 15], btype="bandpass", fs=1000)
overlap = max(len(a), len(b))
print(a)
print(b)

In [ ]:
"""Band-pass Filter"""
ECG_fs = 1000
nyq_rate = ECG_fs / 2

cutoffs = [5,15]
sos = signal.butter(10, cutoffs, btype='bandpass', fs = ECG_fs, output="sos")
a, b = signal.sos2tf(sos)

t,y = signal.impulse((b,a), N=512)
plt.plot(t,y)

In [ ]:
cutoffs = [5,15]
sos = signal.butter(4, cutoffs, btype='bandpass', fs = ECG_fs, output = 'sos')
w, h = signal.sosfreqz(sos, worN=2048, fs = ECG_fs)
plt.plot( w, 20 * np.log10(abs(h)))
plt.xlim((0,100))
plt.ylim(-80,5)
plt.show()

In [ ]:
b = np.convolve(sos[0][:3], sos[1][:3])
a = np.convolve(sos[0][3:], sos[1][3:])
w, h = signal.freqs(b, a)
plt.subplot(2,1,1)
plt.semilogx(w, 20*np.log10(h))
plt.subplot(2,1,2)
plt.semilogx(w, np.unwrap(np.angle(h)))

In [ ]:
t, y = signal.impulse((b,a), T=np.linspace(0, 150, 2**14))
plt.plot(t, y)

In [ ]:
for d in raw_phys.data:
    test = d[1]

In [ ]:
test

In [ ]:
firtest = np.flip(signal.fftconvolve(np.flip(signal.fftconvolve(test, filter_weights, mode="same")),filter_weights,mode="same"))

In [ ]:
test

In [ ]:
filt_data

In [ ]:
filt_data_array = np.squeeze(da.concatenate(filt_data, axis=1))

In [ ]:
type(filt_data_array)

In [ ]:
filt_data